In [1]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.palettes import Category10
import json
import requests
import time
import datetime
import sys
import pandas as pd

output_notebook()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def get_data(url):
    resp = requests.get(url=url)
    data = json.loads(resp.text)
    return data

def data_transform(year, start_date, end_date):
    first_value = df_tickers[year].head(1)["ticker"]
    df_tickers[year]["period"] = df_tickers[year].apply(get_period, year=(year), axis=1)
    df_tickers[year]["change"] = df_tickers[year].apply(get_diff, first_value=(first_value), axis=1)
    
def get_period(df_tickers, year):
    date = df_tickers.date.strftime('%m-%d')
    period = date
    return period
    
def get_diff(df_tickers, first_value):   
    #Adicionando % de diferença entre o valor inicial e o ticker atual
    percentual_diff = ((float(df_tickers.ticker) * 100) / float(first_value)) -100 
    return percentual_diff

Loading BokehJS ...

In [2]:
#Load DataFrames
dates = [["2014","2014-01-01","2014-12-31"], 
         ["2015","2015-01-01","2015-12-31" ], 
         ["2016","2016-01-01","2016-12-31" ],
         ["2017","2017-01-01","2017-12-31"],
         ["2018","2018-01-01","2018-12-31"]]
df_tickers = {}
for date in dates:
    df_tickers[date[0]] = pd.DataFrame.from_dict(get_data('https://api.coindesk.com/v1/bpi/historical/close.json?start='+date[1]+'&end='+date[2])["bpi"], orient='index').reset_index()
    df_tickers[date[0]].columns = ['date', 'ticker']
    df_tickers[date[0]]['date'] = df_tickers[date[0]]['date'].astype('datetime64[ns]')

In [3]:
for date in dates:
    data_transform(date[0], date[1], date[2])
    
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(title="% de Mudança de Valores do Bitcoin por Ano", x_axis_label='year_days', y_axis_label='change', plot_width=950, plot_height=720,  tools=TOOLS)
index = 0

for date in dates:
    p.line(df_tickers[date[0]].index, df_tickers[date[0]].change, line_color=Category10[10][index], legend=date[0], line_width=3)
    index += 1

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)